In [ ]:
from google.colab import drive
drive.mount('/gdrive')
import pathlib

import os

print(os.getcwd())
%cd ..
%cd gdrive
%cd My Drive
%cd MAJOR PROJECT
#data_root = pathlib.Path("/gdrive/My Drive/MAJOR PROJECT")

#%cd ./gdrive/My Drive/MAJOR PROJECT
print(os.getcwd())

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/content
/
/gdrive
/gdrive/My Drive
/gdrive/My Drive/MAJOR PROJECT
/gdrive/My Drive/MAJOR PROJECT


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os
import time

from matplotlib import pyplot as plt
from IPython import display
from tensorflow.keras import layers

In [ ]:
# !pip install -U tensorboard
# !pip install tf-nightly

In [ ]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn import preprocessing


In [ ]:
BUFFER_SIZE = 500
BATCH_SIZE = 1
SHAPE = 128

def load(image_file):
  image = tf.io.read_file(image_file)
  image = tf.image.decode_jpeg(image)

  input_image = tf.cast(image, tf.float32)
  
  return input_image

def resize(input_image, height, width):
  input_image = tf.image.resize(input_image, [height, width],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  

  return input_image

def normalize(input_image):
  input_image = (input_image / 127.5) - 1
  

  return input_image

@tf.function()
def random_jitter(input_image):
  # resizing to 286 x 286 x 3
  input_image = resize(input_image, SHAPE, SHAPE)

  # randomly cropping to 256 x 256 x 3


  return input_image

def load_image_train(image_file):
  input_image= load(image_file)
  input_image= random_jitter(input_image)
  input_image= normalize(input_image)

  return input_image

In [ ]:

trainimg = tf.data.Dataset.list_files(os.getcwd()+'/TRAINS/*.*',shuffle=False)

randomimg = tf.data.Dataset.list_files(os.getcwd()+'/RANDOM/VALID/*.png',shuffle=False)

testimg = tf.data.Dataset.list_files(os.getcwd()+'/img_test/*.*',shuffle=False)


trainimg = trainimg.map(load_image_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

randomimg = randomimg.map(load_image_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

testimg = testimg.map(load_image_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)

random = randomimg.batch(128)
# train = random.batch(128)
# train_dataset = tf.data.Dataset.zip((mri,trainimg))
train = trainimg.batch(1)
# print(train)
# train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# train_dataset = train_dataset.batch(1)
# print(train_dataset)

test = testimg.batch(1)
# print(testimg)
# test_dataset =tf.data.Dataset.zip((mri_test,testimg))
# test_dataset = test_dataset.shuffle(BUFFER_SIZE)
# test_dataset = test_dataset.batch(1)
# print(test_dataset)

In [ ]:
IM=inception.predict(train)
IMT=inception.predict(test)
# feat = tf.data.Dataset.from_tensor_slices(IMT)
# test_dataset =tf.data.Dataset.zip((feat,testimg))
# test_dataset = test_dataset.shuffle(BUFFER_SIZE)
# test_dataset = test_dataset.batch(4)
feat= inception.predict(random)
# print(len(IM))


In [ ]:
import h5py

# h5f = h5py.File('valid4096_data.h5', 'w')
# h5f.create_dataset('train', data=feat)
# h5f.close()
# h5f = h5py.File('train4096_data.h5', 'w')
# h5f.create_dataset('train', data=IM)
# h5f.close()
# h5f = h5py.File('test4096_data.h5', 'w')
# h5f.create_dataset('test', data=IMT)
# h5f.close()
# h5f = h5py.File('validvgg_data.h5','r')
# random = h5f['train'][:]
# h5f.close()
# h5f = h5py.File('train4096_data.h5','r')
# IM = h5f['train'][:]
# h5f.close()
# h5f = h5py.File('testvgg_data.h5','r')
# IMT = h5f['test'][:]
# h5f.close()


In [ ]:
random=tf.data.Dataset.from_tensor_slices(random)
feat_ds = tf.data.Dataset.zip((random,randomimg))
feat_ds = feat_ds.shuffle(BUFFER_SIZE)
feat_ds = feat_ds.batch(64)
print(feat_ds)

In [ ]:

train_feat= tf.data.Dataset.from_tensor_slices(IM)
train_data = tf.data.Dataset.zip((train_feat,trainimg))
train_data = train_data.shuffle(BUFFER_SIZE)
train_data = train_data.batch(16)
print(train_data)



In [ ]:
random=None
feat=None
IM=None
IMT=None
randomimg=None

In [ ]:
def Generator():
    model=tf.keras.Sequential()

    model.add(tf.keras.Input(shape=(8192)))
    model.add(layers.Dense(8192))

    model.add(layers.Reshape((4, 4, 512)))
    assert model.output_shape == (None, 4, 4, 512) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 512)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 64, 64, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 128, 128, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (3, 3), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 128, 128, 3)

    return model

In [ ]:
generator = Generator()
# tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
# generator.summary()


In [ ]:
def Discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[128, 128, 3])) # 64*64*16
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')) # 32*32*64
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    # model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')) # 16*16*128
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same')) # 8*8*256
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same')) # 8*8*256
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = Discriminator()
# discriminator =tf.keras.models.load_model('DISCRIMINATOR')
# tf.keras.utils.plot_model(discriminator, show_shapes=True, dpi=64)

In [ ]:
LAMBDA = 1000

def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss, gan_loss, l1_loss

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_dir = './training_checkpoints8192'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
def generate_images(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(5,5))

  display_list = [ tar[0], prediction[0]]
  title = ['Ground Truth', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
@tf.function
def train_step_1(input_image,target, epoch):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape :

    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator(target, training=True)
    disc_generated_output = discriminator(gen_output, training=True)

    gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)


  generator_gradients = gen_tape.gradient(gen_total_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))
  



In [ ]:
def fit(feat_ds,train_ds,test_ds, epochs):
  for epoch in range(epochs):
    start = time.time()

    display.clear_output(wait=True)


    # Train
    print(epoch)

    for n, (input_image,target) in feat_ds.enumerate():
      print('.', end='')
      if (n+1) % 100 == 0:
        print()
      train_step_1(input_image,target, epoch)
    print()


    for (example_input,example_target) in test_ds.take(1):
        generate_images(generator, example_input, example_target)
    print("TEST Epoch: ", epoch)

    # saving (checkpoint) the model every 10 epochs
    if (epoch + 1) % 10 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))
  # checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
EPOCHS = 50

fit(feat_ds,feat_ds,test_dataset, EPOCHS)

In [ ]:
generator.save('GENERATOR8192')
# discriminator.save('DISCRIMINATOR')

#Neural Network Regression for fMRI decoding

---

Code repository for [Neural Network Regression for fMRI decoding]()

##Requirements
---


*   Python 3.x
*   Tensorflow
*   Numpy
*   Matplotlib
*   H5py
*   Scipy
*   Sklearn
*   Pandas

##Data
---
The required data can be downloaded from the following links:

* [ImageNet validation set]()

* [fMRI data]()

##Setting up files
---
The data files should be places in structure given below:

    Project Folder
        |____ data
                |____fmri
                |     |____subject1.mat
                |     |____imageID_training.csv
                |     |____imageID_test.csv
                |
                |____TRAIN_IMG (training images corresponding to fmri)
                |
                |____TEST_IMG  (testing images corresponding to fmri)
                |
                |____GAN_TRAIN (validation set of imagenet)



##Usage
---
* Install all the mentioned requirements.

* Download necessary data from the given links and place them in given folder structure.

###Extracting VGG19 features

* Run the [vgg19_features.py]() file to extract necessary features. The extracted features are stored in [data]() folder with .h5 extension.

###Training image generating model

* Run [gan_model.py]() for training image generating model. After the script completes, the saved model can be found in saved_models folder.

* This can also be done in step-by-step manner by running the [gan_model.ipynb]() notebook. This is highy recommended for understanding and tinkering with the code.

###Training neural network decoder

* Runnig [decoder.py]() will train the decoder and show the images generated from test fmri data. The default values for decoder variables can be changes before running the script.

* This can also be done in step-by-step manner by running the [decoder.ipynb]() notebook. This is highy recommended for understanding and tinkering with the code.

###Results

The images generated by decoding test fmri data can be found in results folder.


